In [1]:
import fastai
fastai.__version__

'1.0.39'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import pandas as pd
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from fastai import *
from fastai.vision import *
from fastai.vision.image import *
from fastai.callbacks import *
import torchvision
from fastai import basic_train
from torchvision.models.inception import BasicConv2d, InceptionA, InceptionB, InceptionC, InceptionD, InceptionE, InceptionAux

import cv2

from torchvision import models
from pretrainedmodels.models import bninception
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

In [3]:
class DefaultConfigs(object):
    train_data = "./data/train/" # where is your train data
    test_data = "./data/test/"   # your test data
    weights = "./data/checkpoints/"
    best_models = "./data/checkpoints/best_models/"
    submit = "./data/submit/"
    model_name = "bninception_bcelog"
    num_classes = 28
    img_weight = 512
    img_height = 512
    channels = 4
    lr = 0.03
    batch_size = 32
    epochs = 50

config = DefaultConfigs()

In [4]:
def get_net():
    model = bninception(pretrained="imagenet")
    model.global_pool = nn.AdaptiveAvgPool2d(1)
    model.conv1_7x7_s2 = nn.Conv2d(config.channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    model.last_linear = nn.Sequential(
                nn.BatchNorm1d(1024),
                nn.Dropout(0.5),
                nn.Linear(1024, config.num_classes),
            )
    return model

In [5]:
def open_4_channel(fname):
    fname = str(fname)
    suffix = '.png'
    # strip extension before adding color
    if fname.endswith('.png') or fname.endswith('.tif'):
        suffix = fname[-4:]
        fname = fname[:-4]

    colors = ['red','green','blue','yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(fname+'_'+color+suffix, flags).astype(np.float32)/255
           for color in colors]
    
    x = np.stack(img, axis=-1)
    return Image(pil2tensor(x, np.float32).float())

In [6]:
bs = 8

path = Path('data')
df = pd.read_csv(path/'train.csv')

np.random.seed(42)

src = (ImageItemList.from_csv(path, 'train.csv', folder='train', suffix='.png')
       .random_split_by_pct(0.2)
       .label_from_df(sep=' ',  classes=[str(i) for i in range(28)]))

src.train.x.create_func = open_4_channel
src.train.x.open = open_4_channel


src.valid.x.create_func = open_4_channel
src.valid.x.open = open_4_channel

test_ids = list(sorted({fname.split('_')[0] for fname in os.listdir(path/'test')}))

test_fnames = [path/'test'/test_id for test_id in test_ids]

src.add_test(test_fnames, label='0');
src.test.x.create_func = open_4_channel
src.test.x.open = open_4_channel

protein_stats = ([0.08069, 0.05258, 0.05487, 0.08282], [0.13704, 0.10145, 0.15313, 0.13814])
trn_tfms,_ = get_transforms(do_flip=True, flip_vert=True, max_rotate=30., max_zoom=1,
                      max_lighting=0.05, max_warp=0.)

data = (src.transform((trn_tfms, _), size=512).databunch(num_workers=0).normalize(protein_stats))

In [7]:
def _resnet_split(m): return (m[0][6],m[1])
def _default_split(m:nn.Module): return (m[1],)

In [8]:
f1_score = partial(fbeta, thresh=0.2, beta=1)

In [9]:
loss_fn = F.binary_cross_entropy_with_logits
model = get_net()
model = model.cuda()

optim = torch.optim.Adam

learn = basic_train.Learner(data, model, opt_func=optim, loss_func=loss_fn, metrics=[f1_score], 
                           callback_fns=[partial(ReduceLROnPlateauCallback, monitor='trn_loss', patience=3),
                                         partial(EarlyStoppingCallback, monitor='trn_loss', min_delta=0.01, patience=6)])
learn.unfreeze()

In [10]:
learn.fit(50, 5e-3)

epoch,train_loss,valid_loss,fbeta
1,0.178510,0.169843,0.356882
2,0.170537,0.167872,0.361936
3,0.156742,0.454099,0.336654
4,0.144803,0.141443,0.473839
5,0.137286,0.139061,0.488699
6,0.128798,0.126561,0.559848
7,0.123195,0.146225,0.500717
8,0.117466,0.124168,0.561127
9,0.114366,0.119588,0.581671
10,0.109713,0.109103,0.620714


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 24: early stopping


In [11]:
learn.save("early_stopped_bn_inception")

In [12]:
preds, avg_pred, y = learn.TTA(beta=None, scale=1.05, ds_type=DatasetType.Test)

pred_labels = [' '.join(list([str(i) for i in np.nonzero(row > 0.2)[0]])) for row in np.array(avg_pred)]
df = pd.DataFrame({'Id':test_ids,'Predicted':pred_labels})
df.to_csv(path/'bn_inception-avg_pred-0.2.csv', header=True, index=False)

pred_labels = [' '.join(list([str(i) for i in np.nonzero(row > 0.2)[0]])) for row in np.array(preds)]
df = pd.DataFrame({'Id':test_ids,'Predicted':pred_labels})
df.to_csv(path/'bn_inception-preds_v1-0.2.csv', header=True, index=False)

In [13]:
pred_labels = [' '.join(list([str(i) for i in np.nonzero(row > 0.15)[0]])) for row in np.array(avg_pred)]
df = pd.DataFrame({'Id':test_ids,'Predicted':pred_labels})
df.to_csv(path/'bn_inception-avg_pred-0.1.csv', header=True, index=False)

pred_labels = [' '.join(list([str(i) for i in np.nonzero(row > 0.15)[0]])) for row in np.array(preds)]
df = pd.DataFrame({'Id':test_ids,'Predicted':pred_labels})
df.to_csv(path/'bn_inception-preds_v1--0.1.csv', header=True, index=False)

In [14]:
learn = basic_train.Learner(data, model, opt_func=optim, loss_func=loss_fn, metrics=[f1_score], 
                           callback_fns=[partial(ReduceLROnPlateauCallback, monitor='val_loss', patience=3),
                                         partial(EarlyStoppingCallback, monitor='val_loss', min_delta=0.01, patience=6)])
learn.unfreeze()

In [15]:
learn.load("early_stopped_bn_inception")

Learner(data=ImageDataBunch;

Train: LabelList
y: MultiCategoryList (24858 items)
[MultiCategory 16;0, MultiCategory 7;1;2;0, MultiCategory 5, MultiCategory 1, MultiCategory 18]...
Path: data
x: ImageItemList (24858 items)
[Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512)]...
Path: data;

Valid: LabelList
y: MultiCategoryList (6214 items)
[MultiCategory 12;23;0, MultiCategory 0, MultiCategory 1;0, MultiCategory 25;5, MultiCategory 23]...
Path: data
x: ImageItemList (6214 items)
[Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512)]...
Path: data;

Test: LabelList
y: MultiCategoryList (11702 items)
[MultiCategory 0, MultiCategory 0, MultiCategory 0, MultiCategory 0, MultiCategory 0]...
Path: data
x: ImageItemList (11702 items)
[Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512), Image (4, 512, 512)]...
Path: data, model=BNInception(
  (conv1_7x7_s2): 

In [ ]:
learn.fit(10, 1e-4)

epoch,train_loss,valid_loss,fbeta
1,0.081434,0.080142,0.730086
2,0.078901,0.079262,0.733555
3,0.078889,0.078498,0.736383
4,0.078353,0.078389,0.738169
5,0.080327,0.077908,0.739681
6,0.077251,0.077787,0.739039
